In [1]:
import random
from math import gcd

class OpenKey:
    def __init__(self):
        self.clear_alphabet = "abcdefghijklmnopqrstuvwxyzåäö ?!%/&.,1234567890"
        self.prime_length = 2048 
        self.prime1 = 0
        self.prime2 = 0
        self.N = 0
        self.E = 0
        self.fi_PQ = 0
        self.D = 0

    def get_number_from_message(self, message):
        # Use a dictionary for faster character lookup
        char_to_num = {char: str(index + 10) for index, char in enumerate(self.clear_alphabet)}
        numbers = [char_to_num.get(char, '') for char in message]
        return [int(num) for num in numbers if num]

    def get_random_prime(self):
        while True:
            candidate = random.getrandbits(self.prime_length)
            candidate |= 1
            if self.is_prime(candidate):
                return candidate

    def is_prime(self, n, k=5):
        if n <= 1:
            return False
        if n <= 3:
            return True
        if n % 2 == 0:
            return False

        # Run the Miller-Rabin primality test k times
        for _ in range(k):
            a = random.randint(2, n - 2)
            x = pow(a, n - 1, n)
            if x != 1:
                return False

        return True

    def get_message_from_number(self, numbers):
        return ''.join([self.clear_alphabet[num - 10] for num in numbers])

    def mod_inverse(self, a, n):
        # Find modular inverse
        k = 1
        while (k * self.fi_PQ + 1) % self.E != 0:
            k += 1
        v = (k * self.fi_PQ + 1) // self.E
        return v

    def generate_key_pair(self):
        self.prime1 = self.get_random_prime()
        self.prime2 = self.get_random_prime()

        while self.prime1 == self.prime2:
            self.prime2 = self.get_random_prime()

        self.N = self.prime1 * self.prime2
        self.fi_PQ = (self.prime1 - 1) * (self.prime2 - 1)

        # Find a suitable public exponent (E) that is relatively prime to fi_PQ
        self.E = 65537  # Common choice for E
        while not self.is_relative_prime(self.E, self.fi_PQ):
            self.E += 1

        # Calculate the private exponent (D)
        self.D = self.mod_inverse(self.E, self.fi_PQ)

    def encrypt(self, message):
        un = self.get_number_from_message(message)
        msg = ''.join(map(str, un))
        encrypted_msg = str(self.divide(int(msg), self.E))
        return encrypted_msg

    def divide(self, msg_in_int, x):
        value = 1
        big = msg_in_int
        abcd = [int(item) for item in str(x)[::-1]]
        r = True
        for item in abcd:
            if r:
                r = False
            else:
                big = pow(big, 10, self.N)
            value *= pow(big, item, self.N)
        return value % self.N

    def decrypt(self, encrypted_msg):
        ro = self.divide(int(encrypted_msg), self.D)
        decrypted = str(ro)
        k = [int(decrypted[i:i+2]) for i in range(0, len(decrypted), 2)]
        return self.get_message_from_number(k)

    def is_relative_prime(self, a, b):
        return gcd(a, b) == 1

In [2]:
if __name__ == "__main__":
    open_key = OpenKey()
    open_key.generate_key_pair()
    print(f"Generated Public Key (N, E): ({open_key.N}, {open_key.E}) \n")
    print(f"Generated Private Key (D): {open_key.D} \n")

    message = "your text"
    encrypted_msg = open_key.encrypt(message)
    print(f"Encrypted Message: ({encrypted_msg}) \n")

    decrypted_msg = open_key.decrypt(encrypted_msg)
    print(f"Decrypted Message: {decrypted_msg}")

Generated Public Key (N, E): (1447785130365800592551907786310142302240556366304579076805666628633206909960840672834116821354410908733402355062375853647517059871725020915114454836583517283356922714905890736636403939864301669951798464707752369322711083852868769639103913533896239700905009018799471134857587152132111090318275432809082910593495049695376987875502343685266227190647011702182240333675191436526893707298686077619370057749001534125867446841247899243798653484644034147776084566063562847387218378811156902590521364355707946197843552268311801499322250753374037836208041821897167704979894701631727450918447921219958692094465627981182860415437985876740441951315437889650899347800169149873363795645575668055248546233687925822012718898999670591143599755430740927514139160109779570751727379921039167157753434320704596430467216698192601037562870124631491362885700543600942005385542213659516865055054742031450296232180122341169655260041021921530579151855046144914728508389799663433406035211085478191978